import numpy and sepshow the first 10 objects results:

In [ ]:
import numpy as np
import sep

additional setup for reading the test image and displaying plots

In [ ]:
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib import rcParams

%matplotlib inline

rcParams['figure.figsize'] = [10.0, 8.0]

open the FITS file & read data

In [ ]:
fname = "f105w.fits"
hdu_list = fits.open(fname)
hdu_list.info()
data = hdu_list[0].data
data = data.byteswap(inplace=True).newbyteorder()

read image into standard 2-d numpy array & show the image

In [ ]:
print(type(data))
print(data.shape)
image_header = hdu_list[0].header
hdu_list.close()

m = np.mean(data)
s = np.std(data)
fig1 = plt.figure(figsize=(10.0,8.0))
plt.imshow(data, interpolation = 'nearest', cmap='gray',vmin=m-s,vmax=m+s,origin='lower')
plt.colorbar()
fig1.savefig('data.png')

 measure a spatially varying background on the image

In [ ]:
bkg = sep.Background(data)

get a "global" mean and noise of the image background:

In [ ]:
print(bkg.globalback)
print(bkg.globalrms)

evaluate background as 2-d array, same size as original image

In [ ]:
bkg_image = bkg.back()


show the background

In [ ]:
fig2 = plt.figure(figsize=(10.0,8.0))
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
fig2.savefig('Background.png')

evaluate the background noise as 2-d array, same size as original image

In [ ]:
bkg_rms = bkg.rms()

show the background noise

In [ ]:
fig3 = plt.figure(figsize=(10.0,8.0))
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
fig3.savefig('Backgroundnoise.png')

Subtract the background

In [ ]:
data_sub = data - bkg

Object Detection

In [ ]:
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)

how many objects were detected

In [ ]:
len(objects)

plot background-subtracted image

In [ ]:
from matplotlib.patches import Ellipse


fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray',
               vmin=m-s, vmax=m+s, origin='lower')



plot an ellipse for each object

In [ ]:
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)
    fig.savefig('sources_found_f105w.png')

available fields

In [ ]:
objects.dtype.names

show the first 10 objects results:

In [ ]:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))